In [1]:
import sys # for automation and parallelization: set manual to false when run by a launcher
import json
 
default = {'scenario': 'houston','project':'houston','training_folder':'../../scenarios'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
import numpy as np
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [3]:
scenario = argv['scenario']

base_folder = os.path.join(argv['training_folder'],argv['scenario'])
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(base_folder, 'model/')


In [4]:
base_folder

'../../scenarios/houston'

In [5]:
sm = stepmodel.read_zippedpickles(model_folder +'aon_los')

zones: 100%|██████████| 4/4 [00:00<00:00, 43.07it/s]


In [6]:

var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario=scenario, return_ancestry=False) 

motor_rate = var['generation']['motor_rate'] 
pt_rate = (1-motor_rate)


In [7]:
pop_mobility = var['generation']['pop_mobility']
job_mobility = var['generation']['job_mobility']

sm.zones['emission'] =  sm.zones['population'] * pop_mobility 
sm.zones['attraction'] =  sm.zones['jobs'] *job_mobility

car = sm.copy()
car.zones['attraction']= car.zones['attraction'] #*pop_mobility
car.zones['emission'] *= motor_rate  

pt_captive = sm.copy()
pt_captive.zones['emission']= pt_captive.zones['emission']
pt_captive.zones['emission'] *= pt_rate

In [8]:
power = var['step_distribution']['matrix_power_car']

imp_matrix = car.car_los[ 
    ['origin', 'destination', 'time']
].set_index(['origin','destination']).unstack()
imp_matrix = imp_matrix.replace(0, var['step_distribution']['time_intrazonal'])

imp_matrix = imp_matrix.pow(power)

car.step_distribution(deterrence_matrix=1/imp_matrix) 

In [9]:
power = var['step_distribution']['matrix_power_pt']

imp_matrix = pt_captive.pt_los[ 
    ['origin', 'destination', 'gtime']
].set_index(['origin','destination']).unstack()
imp_matrix = imp_matrix.replace(0, var['step_distribution']['time_intrazonal']).fillna(1e16)

imp_matrix = imp_matrix.pow(power)

pt_captive.step_distribution(deterrence_matrix=1/imp_matrix)



In [10]:
imp_matrix

gtime                                            \
destination        zone_0        zone_1       zone_10      zone_100   
origin                                                                
zone_0       7.062794e+03  1.000000e+20  1.000000e+20  1.000000e+20   
zone_1       1.000000e+20  7.062794e+03  1.000000e+20  1.000000e+20   
zone_10      1.000000e+20  1.000000e+20  7.062794e+03  1.000000e+20   
zone_100     1.000000e+20  1.000000e+20  1.000000e+20  7.062794e+03   
zone_101     1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
...                   ...           ...           ...           ...   
zone_95      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_96      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_97      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_98      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_99      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   

                                                                     \
destination      zone_101      zone_102      zone_103      zone_104   
origin                                                                
zone_0       1.000000e+20  1.000000e+20  1.000000e+20  6.191004e+04   
zone_1       1.000000e+20  1.000000e+20  1.000000e+20  9.925205e+04   
zone_10      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_100     1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_101     7.062794e+03  1.000000e+20  1.000000e+20  1.000000e+20   
...                   ...           ...           ...           ...   
zone_95      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_96      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_97      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_98      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_99      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   

                                         ...                              \
destination      zone_105      zone_106  ...       zone_90       zone_91   
origin                                   ...                               
zone_0       1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
zone_1       1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
zone_10      1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
zone_100     1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
zone_101     1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
...                   ...           ...  ...           ...           ...   
zone_95      1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
zone_96      1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
zone_97      1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
zone_98      1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   
zone_99      1.000000e+20  1.000000e+20  ...  1.000000e+20  1.000000e+20   

                                                                     \
destination       zone_92       zone_93       zone_94       zone_95   
origin                                                                
zone_0       1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_1       1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_10      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_100     1.000000e+20  1.088855e+05  1.000000e+20  1.000000e+20   
zone_101     1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
...                   ...           ...           ...           ...   
zone_95      1.000000e+20  1.000000e+20  1.000000e+20  7.062794e+03   
zone_96      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_97      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_98      1.000000e+20  1.000000e+20  1.000000e+20  1.000000e+20   
zone_99      1.000000e+20  1.000000e+20  1

In [11]:
sm.volumes = pd.merge(
    car.volumes,
    pt_captive.volumes,
    on=['origin', 'destination'],
    suffixes=[ '_car_owner', '_pt_captive']
).rename(columns={'volume_car_owner': 'car_owner', 'volume_pt_captive': 'pt_captive'})


In [12]:
sm.sample_volumes(bidimentional_sampling=True, fit_sums=True, sample_weight={'car_owner':50, 'pt_captive':50}, sample_size=None, inplace=True)

In [13]:
if manual:
    
    od_folder =  os.path.join(input_folder,'od/')

    od = od_folder + 'od.geojson'

    od_test = gpd.read_file(od)
    if 'name' not in od_test.columns:
        od_test['name'] = od_test['index']
    od_test['name'] = od_test['name'].fillna(od_test['index'].astype(str))
    from shapely.geometry import Point
    from syspy.spatial.spatial import nearest, agglomerative_clustering, voronoi_diagram_dataframes, add_geometry_coordinates
    od_test['geometry_o'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][0]))
    od_test['geometry_d'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][1]))
    # find nearest node with KNN. nodes are now the origin and destination.
    od_test['geometry'] = od_test['geometry_o']
    neigh = nearest(od_test, sm.zones.to_crs(4326), n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    od_test['origin'] = od_test.index.map(zone_node_dict.get)

    od_test['geometry'] = od_test['geometry_d']
    neigh = nearest(od_test, sm.zones.to_crs(4326), n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    od_test['destination'] = od_test.index.map(zone_node_dict.get)

    od_test = od_test.drop(columns=['geometry_o','geometry_d','geometry'])
    od_test["pt_captive"] = 0.001
    #Add od_test to volumes after sparsification
    sm.volumes = pd.concat([sm.volumes,od_test[["origin","destination","pt_captive"]]]).sort_values(by="pt_captive",ascending = False).drop_duplicates(["origin","destination"])


    sm.volumes= pd.concat([sm.volumes,od_test[["origin","destination","pt_captive"]]]).sort_values(by="pt_captive",ascending = False).drop_duplicates(["origin","destination"])

In [14]:
sm.to_zippedpickles(model_folder +'distribution', only_attributes= ['volumes', 'zones_before_clustering'])


volumes: 100%|██████████| 11/11 [00:00<00:00, 31.14it/s]               


In [15]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined